In [1]:
import wooldridge as woo
import pandas as pd

from statsmodels.formula.api import ols
from statsmodels.formula.api import glm

import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [10, 5]


# Interactive Session 12

In [2]:
bwght=woo.data('bwght')
woo.data('bwght',description=True)

name of dataset: bwght
no of variables: 14
no of observations: 1388

+----------+--------------------------------+
| variable | label                          |
+----------+--------------------------------+
| faminc   | 1988 family income, $1000s     |
| cigtax   | cig. tax in home state, 1988   |
| cigprice | cig. price in home state, 1988 |
| bwght    | birth weight, ounces           |
| fatheduc | father's yrs of educ           |
| motheduc | mother's yrs of educ           |
| parity   | birth order of child           |
| male     | =1 if male child               |
| white    | =1 if white                    |
| cigs     | cigs smked per day while preg  |
| lbwght   | log of bwght                   |
| bwghtlbs | birth weight, pounds           |
| packs    | packs smked per day while preg |
| lfaminc  | log(faminc)                    |
+----------+--------------------------------+

J. Mullahy (1997), “Instrumental-Variable Estimation of Count Data
Models: Applications to Models of C

In [3]:
bwght['l_bwght']=np.log(bwght['bwght'])
bwght['l_faminc']=np.log(bwght['faminc'])

In [4]:
form1='l_bwght~cigs+l_faminc+parity+male+white+motheduc+fatheduc'
mod1=ols(form1,data=bwght).fit()
print(mod1.summary())
print('As in the table above aic and bic',mod1.aic,mod1.bic)

                            OLS Regression Results                            
Dep. Variable:                l_bwght   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     8.764
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           1.55e-10
Time:                        09:19:30   Log-Likelihood:                 331.11
No. Observations:                1191   AIC:                            -646.2
Df Residuals:                    1183   BIC:                            -605.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6527      0.038    121.943      0.0

In [5]:
restrictions='(motheduc=0),(fatheduc=0)'
mod1.f_test(restrictions)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.85152619]]), p=0.4270245171619057, df_denom=1.18e+03, df_num=2>

In [6]:
#Some of the data points were dropped in the orginal equation because of missing values for education. 
#So to compare the models we need to drop thos with missing values
bwght_=bwght.copy()
bwght_=bwght_[['l_bwght','cigs','l_faminc','parity','male','white','motheduc','fatheduc']]
bwght_=bwght_.dropna(axis=0)

In [7]:
form2='l_bwght~cigs+l_faminc+parity+male+white'
mod2=ols(form2,bwght_).fit()
print(mod2.summary())
print('As in the table above aic and bic',mod2.aic,mod2.bic)

                            OLS Regression Results                            
Dep. Variable:                l_bwght   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     11.93
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           2.75e-11
Time:                        09:19:30   Log-Likelihood:                 330.25
No. Observations:                1191   AIC:                            -648.5
Df Residuals:                    1185   BIC:                            -618.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6518      0.030    155.956      0.0

In [8]:
#The easiest way to impose equality restrictions is
form1_='l_bwght~cigs+l_faminc+parity+male+white+I(motheduc+fatheduc)'
mod1_=ols(form1_,data=bwght).fit()
print(mod1_.summary())
print('As in the table above aic and bic',mod1_.aic,mod1_.bic)

                            OLS Regression Results                            
Dep. Variable:                l_bwght   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     9.953
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           9.70e-11
Time:                        09:19:30   Log-Likelihood:                 330.30
No. Observations:                1191   AIC:                            -646.6
Df Residuals:                    1184   BIC:                            -611.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  4

In [9]:
#You can  also test and impose restrictions using glm
restrictions='motheduc=fatheduc'
print(mod1.f_test(restrictions))
mod3=glm(form1,data=bwght_)
out=mod3.fit_constrained(restrictions)
print(out.summary())
print('aic and bic',out.aic,out.bic_llf)

<F test: F=array([[1.60260064]]), p=0.20578429052760874, df_denom=1.18e+03, df_num=1>
                 Generalized Linear Model Regression Results                  
Dep. Variable:                l_bwght   No. Observations:                 1191
Model:                            GLM   Df Residuals:                     1184
Model Family:                Gaussian   Df Model:                            6
Link Function:               identity   Scale:                        0.033822
Method:                          IRLS   Log-Likelihood:                 330.30
Date:                Sat, 07 Jan 2023   Deviance:                       40.045
Time:                        09:19:31   Pearson chi2:                     40.0
No. Iterations:                     1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------

# Session 12 Slides

In [10]:
data=pd.DataFrame(np.random.random([3,3]))
data.columns=['Y','X','Z']

In [11]:
mod=ols('Y~X+Z',data=data).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 07 Jan 2023   Prob (F-statistic):                nan
Time:                        09:19:31   Log-Likelihood:                 100.45
No. Observations:                   3   AIC:                            -194.9
Df Residuals:                       0   BIC:                            -197.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3192        inf         -0        n

C:\Users\aes05kgb\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\aes05kgb\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\aes05kgb\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\aes05kgb\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [12]:
mod=ols('Y~X',data=data).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                 -0.293
Method:                 Least Squares   F-statistic:                    0.5473
Date:                Sat, 07 Jan 2023   Prob (F-statistic):              0.595
Time:                        09:19:31   Log-Likelihood:                0.28163
No. Observations:                   3   AIC:                             3.437
Df Residuals:                       1   BIC:                             1.634
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1294      0.936     -0.138      0.9

C:\Users\aes05kgb\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [13]:
data=pd.read_csv('data9-5.csv')
names=data.columns[1:]
lnames=['obs']
for i in names:
    j='l_'+i
    lnames=lnames+[j]
ldata=np.log(data)
ldata['obs']=data['obs']
ldata.columns=lnames
ldata['time']=np.arange(1,len(data)+1)


In [14]:
#Lets automate the construction of the formula
formula='l_output~'
for i in ldata.columns[2:]:
    formula=formula+'+'+i
formula


'l_output~+l_labor+l_land+l_machines+l_energy+l_fert+l_seedfeed+l_others+time'

In [15]:
mod1=ols(formula,data=ldata).fit()
print(mod1.summary())

                            OLS Regression Results                            
Dep. Variable:               l_output   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     368.8
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           7.25e-33
Time:                        09:19:31   Log-Likelihood:                 100.87
No. Observations:                  46   AIC:                            -183.7
Df Residuals:                      37   BIC:                            -167.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2850      1.333     -0.214      0.8

In [16]:
hypotheses = '(l_labor= 0),(l_machines = 0),(l_energy= 0),(l_fert=0),(l_seedfeed=0),(l_others=0)'
f_test = mod1.f_test(hypotheses)
print(f_test)

<F test: F=array([[0.96114115]]), p=0.4646388214508299, df_denom=37, df_num=6>


In [17]:
mod2=ols('l_output~l_land+time',data=ldata).fit()
print(mod2.summary())

                            OLS Regression Results                            
Dep. Variable:               l_output   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     1480.
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           2.24e-40
Time:                        09:19:32   Log-Likelihood:                 97.541
No. Observations:                  46   AIC:                            -189.1
Df Residuals:                      43   BIC:                            -183.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7266      0.615      2.810      0.0

In [18]:
#Produces only the aic and bic as below - not the Hannan-Quinn
[[mod1.aic,mod1.bic],[mod2.aic,mod2.bic]]

[[-183.7456596860726, -167.28788711767075],
 [-189.08277656361366, -183.59685237414638]]

In [19]:
hypotheses = '(l_labor+l_land+l_machines+l_energy+l_fert+l_seedfeed+l_others=1)'
f_test = mod1.f_test(hypotheses)
print(f_test)

<F test: F=array([[0.17219901]]), p=0.6805612639030256, df_denom=37, df_num=1>


In [20]:
hypotheses = '(l_labor+l_land+l_machines+l_energy+l_fert+l_seedfeed+l_others=1),(l_machines=l_others),(l_energy=l_others),(l_fert=l_others),(l_seedfeed=l_others)'
f_test = mod1.f_test(hypotheses)
print(f_test)

<F test: F=array([[0.72626047]]), p=0.6081363146168255, df_denom=37, df_num=5>


In [21]:
#In order to impose non-exclusion linear restrictions we need to use glm
#Swapping ols with glm we can get the same results as ols
mod11 = glm(formula =formula, data = ldata)
#print(mod11.fit().summary())

#But now we can impose restrictions
modl_rls = mod11.fit_constrained(hypotheses)
print(modl_rls.summary().tables[1])
print('aic and bic',modl_rls.aic,modl_rls.bic_llf)

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0000      0.072    -13.805      0.000      -1.142      -0.858
l_labor        0.3549      0.098      3.615      0.000       0.163       0.547
l_land         0.5403      0.177      3.056      0.002       0.194       0.887
l_machines     0.0209      0.018      1.145      0.252      -0.015       0.057
l_energy       0.0209      0.018      1.145      0.252      -0.015       0.057
l_fert         0.0209      0.018      1.145      0.252      -0.015       0.057
l_seedfeed     0.0209      0.018      1.145      0.252      -0.015       0.057
l_others       0.0209      0.018      1.145      0.252      -0.015       0.057
time           0.0275      0.002     13.851      0.000       0.024       0.031
aic and bic -189.43910055416998 -182.1245349682136


In [22]:
ldata['allothers']=(ldata['l_machines']+ldata['l_energy']+ldata['l_fert']+ldata['l_seedfeed']+ldata['l_others'])/5
ldata['y']=ldata['l_output']-ldata['allothers']
ldata['lbd']=ldata['l_labor']-ldata['allothers']
ldata['landd']=ldata['l_land']-ldata['allothers']

In [23]:
mod12 = ols(formula ='y~lbd+landd+time', data = ldata).fit()
print(mod12.summary())
print('\n As in the table above The AIC and BIC are', mod12.aic, mod12.bic)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.939
Method:                 Least Squares   F-statistic:                     232.3
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           3.62e-26
Time:                        09:19:33   Log-Likelihood:                 98.720
No. Observations:                  46   AIC:                            -189.4
Df Residuals:                      42   BIC:                            -182.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0000      0.072    -13.805      0.0

In [24]:
mod13 = ols(formula ='l_output~l_land+time', data = ldata).fit()
print(mod13.summary())
print('\n As in the table above The AIC and BIC are', mod13.aic, mod13.bic)

                            OLS Regression Results                            
Dep. Variable:               l_output   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     1480.
Date:                Sat, 07 Jan 2023   Prob (F-statistic):           2.24e-40
Time:                        09:19:33   Log-Likelihood:                 97.541
No. Observations:                  46   AIC:                            -189.1
Df Residuals:                      43   BIC:                            -183.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7266      0.615      2.810      0.0